In [ ]:
from cedne import utils
from cedne import NervousSystem, Connection 
import matplotlib.pyplot as plt
import numpy as np
import scanpy as sp
import pickle

In [ ]:
f = utils.makeFly()
nn = f.networks["Neutral"]

In [ ]:
nlist, npos = [], []
for n in nn.neurons:
    nlist.append(n) 
    npos.append(nn.neurons[n].position)

npos = np.array(npos)

In [ ]:
l0, l1, l2 = [], [], []
for n in nlist:
    l_ = n.split(".")
    if l_[0] == "LO":
        l0.append(n)
        if len(l_)>2:
            l1.append(l_[1])
            l2.append(l_[2])
        else:
            l1.append('')
            l2.append(l_[1])

In [ ]:
from collections import Counter
thresL1 = [k for k, v in Counter(l1).items() if v>00]

In [ ]:
l_subsets = thresL1 #list(set(l1))
cols_ind = []
alpha_ind= []
for n in nlist:
    if n in l0 and l1[l0.index(n)] in l_subsets:
        cols_ind.append(l_subsets.index(l1[l0.index(n)]))
        alpha_ind.append(0)
    else:
        cols_ind.append(len(l_subsets))
        alpha_ind.append(1)

In [ ]:
l_subsets

In [ ]:
colors = plt.cm.tab10(np.arange(len(l_subsets)))
colors = np.append(colors, [[0.5,0.5,0.5,1]], axis=0)
alpha_arr = [1,3e-2]

cols = colors[cols_ind]
alpha = [alpha_arr[a] for a in alpha_ind]
labels = [l_subsets[j] if j<len(l_subsets) else 'Other' for j in cols_ind]

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(6,6), layout='constrained')
ax.set_aspect('equal')
scatter = ax.scatter(npos[:,0], npos[:,1], s=1, c=cols, alpha=alpha, label=labels)
# legend1 = f.legend(*scatter.legend_elements(num=2),
#                     loc="outside upper center", fontsize=72)
# f.add_artist(legend1)
ax.axis('off')
plt.savefig("./fly-LO1.png")
plt.close()

## Subnetwork by creating a fresh network with neurons and edges. 

In [ ]:
nn_l0 = NervousSystem(worm = f, network = 'subnet')
ntype, npos, nlen, narea, nvol = zip(*[(nn.neurons[n].type, nn.neurons[n].position, nn.neurons[n].length, nn.neurons[n].area, nn.neurons[n].volume) for n in l0])

In [ ]:
nn_l0.create_neurons(labels=l0, type=ntype, position=npos, length=nlen, area=narea, volume=nvol)

for e in nn.connections.keys():
    if e[0].name in l0 and e[1].name in l0:
        conn_id = (nn_l0.neurons[e[0].name], nn_l0.neurons[e[1].name], e[2])
        conn_data = nn.connections[e].neurotransmitter
        nn_l0.connections.update( { conn_id: Connection(conn_id[0], conn_id[1], conn_id[2], neurotransmitter=conn_data) })

In [ ]:
classes = {}
for n in nn_l0.neurons:
    if not nn_l0.neurons[n].type in classes:
        classes[nn_l0.neurons[n].type] = []
    classes[nn_l0.neurons[n].type].append(n)

In [ ]:
cat_subnet = nn_l0.fold_network(classes, data='clean')

In [ ]:
cat_subnet.remove_unconnected_neurons()

In [ ]:
cat_subnet_filt = cat_subnet.subnetwork([n for n in cat_subnet.neurons if n=='LO' or n.split('.')[1] in l_subsets], as_view=True)

In [ ]:
[n for n in cat_subnet_filt.neurons]

In [ ]:
fig_ent = utils.plot_shell(cat_subnet_filt, save='fly_LO1.svg', shells=[[], [cat_subnet_filt.neurons[n] for n in cat_subnet_filt.neurons]], figsize=(8,8), width_logbase=10)

In [ ]:
[n for n in cat_subnet.neurons], l_subsets

In [ ]:
node_color = {}
#for n in cat_subnet.neurons:
for n in cat_subnet.neurons:
    nsplit = n.split('.')
    if len(nsplit)>1:
        node_color[cat_subnet.neurons[n]] = colors[l_subsets.index(nsplit[1])]
    else:
        node_color[cat_subnet.neurons[n]] = colors[l_subsets.index('')]

In [ ]:
nt_conn = {}
for e, conn in cat_subnet.connections.items():
    nt_conn[e] = set([conn.contraction_data[k].neurotransmitter['neurotransmitter'] for k in conn.contraction_data.keys()])

In [ ]:
dop_conn

In [ ]:
nname = 'LO.LOP'
dop_conn = {k:'navy' if 'DA' in nt_conn[k] and (k[0].name == nname or k[1].name == nname) else 'lightgray' for k in nt_conn}

In [ ]:
fig_ent = utils.plot_shell(cat_subnet, save='fly_LO_color.svg', shells=[[], [cat_subnet.neurons[n] for n in cat_subnet.neurons]], figsize=(8,8), width_logbase=10, edge_color_dict=dop_conn, node_color_dict=node_color)

In [ ]:
thresNeurs

In [ ]:
with open("Fly_LO_grouped.pkl", 'wb') as pickle_file:
    pickle.dump(cat_subnet, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
loom = sp.read_10x_h5("/Users/sahilmoza/Downloads/s_fca_biohub_head_10x.loom")

In [ ]:
with lp.connect("/Users/sahilmoza/Downloads/s_fca_biohub_head_10x.loom") as ds:
    # Get expression values for a specific gene
    gene_expression = ds[ds.ra.Gene == "Gene1", :] 
    print(gene_expression)

In [ ]:
f.save("./fly.pkl")

In [ ]:
mb0, mb1, mb2 = [], [], []
for n in nlist:
    if "MB" in n:
        l_ = n.split(".")
        mb0.append(n)
        if len(l_)>2:
            mb1.append(l_[1])
            mb2.append(l_[2])
        else:
            mb1.append('')
            mb2.append(l_[1])

In [ ]:
nlist, npos = [], []
for n in nn.neurons:
    nlist.append(n) 
    npos.append(nn.neurons[n].position)

npos = np.array(npos)

In [ ]:
mb_subsets = list(set(mb1))
cols_ind = []
alpha_ind= []
for n in nlist:
    if n in mb0:        
        cols_ind.append(mb_subsets.index(mb1[mb0.index(n)]))
        alpha_ind.append(0)
    else:
        cols_ind.append(len(mb_subsets))
        alpha_ind.append(1)

In [ ]:
colors = plt.cm.tab20(np.arange(len(mb_subsets)))
colors = np.append(colors, [[0.5,0.5,0.5,1]], axis=0)
alpha_arr = [1,3e-2]

cols = colors[cols_ind]
alpha = [alpha_arr[a] for a in alpha_ind]
labels = [mb_subsets[j] if j<len(mb_subsets) else 'Other' for j in cols_ind]

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(6,6), layout='constrained')
ax.set_aspect('equal')
scatter = ax.scatter(npos[:,0], npos[:,1], s=1, c=cols, alpha=alpha, label=labels)
# legend1 = f.legend(*scatter.legend_elements(num=2),
#                     loc="outside upper center", fontsize=72)
# f.add_artist(legend1)
ax.axis('off')
plt.savefig("./fly-MB.png")
plt.close()

In [ ]:
npos[:,0]